<!--NAVIGATION-->
< [Calculate Flow Direction](Flowdirection.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Catchment Delineation](CatchmentDelineation.ipynb) >

# 6. Délimitation de cours d'eau
Dans cette étape, nous allons délimiter les cours d'eau. Nous appliquerons deux méthodes:
* Utilisation des ordres de Strahler
* Utilisation de l'accumulation de flux (écoulement)

## 6.1 Méthode de Strahler
Commençons par dériver les ordres de Strahler en utilisant l'opération <code>streamorder</code>.
L'opération <code>streamorder</code> nécessite comme entrée, la carte de direction du flux créée dans la section précédente. Une description détaillée de l'opération <code>streamorder</code> peut être trouvée dans la [documentation de PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_streamorder.html).

In [ ]:
import os
from pcraster import *
os.chdir('./Data/')

FlowDirection = readmap('flowdir.map')
StrahlerOrders = streamorder(FlowDirection)
aguila(StrahlerOrders)

Changez la légende dans Aguila en une rampe allant du jaune au bleu, de sorte que les ordres supérieurs soient bleus, comme les rivières.

Le type de données de la carte d'ordre de Strahler est ordinal, commençant à 1. Maintenant, nous devons déterminer après quel ordre de Strahler nous considérons que le flux est suffisamment important pour l'appeler une rivière. Nous faisons cela par le biais de la calibration. Calculons des cartes avec un ordre de Strahler de 1 au maximum et comparons-les avec OpenStreetMap.
Nous déterminons d’abord l'ordre de Strahler maximum en utilisant l'opération <code>mapmaximum</code>. L'opération <code>mapmaximum</code> est une opération globale. Plus d'informations peuvent être trouvées dans la [documentation de PCRaster ](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_mapmaximum.html).

In [ ]:
MaximumStrahlerOrder = mapmaximum(StrahlerOrders)

Visualiser le résultat avec Aguila.

Le problème est que le résultat est une carte PCRaster avec pour chaque pixel la valeur maximale de la carte. . Afin d'itérer sur les ordres de Strahler, nous avons besoin d'obtenir cette valeur. Nous pouvons utiliser l'opération <code>cellvalue</code>. L'opération <code>cellvalue</code> nécessite le raster  qui donne la valeur de la cellule en entrée ainsi qu'un numéro d'index de ligne et de colonne. Vu qu'ici, toutes les cellules sont les mêmes, nous pouvons donc simplement utiliser 0 pour l'index, en nous référant à la première ligne et à la première colonne du raster d'entrée.

In [ ]:
MaximumStrahlerOrderTuple = cellvalue(MaximumStrahlerOrder,0,0)
print(MaximumStrahlerOrderTuple)

L'opération <code>cellvalue</code> renvoie un tuple avec deux éléments : le premier est la valeur de la cellule, le second est une valeur booléenne qui indique si le premier élément, est valide ou non. Si le deuxième élément est False, la cellule contient une valeur manquante. Donc si nous voulons avoir la valeur, nous devons prendre le premier élément du tuple.

In [ ]:
MaximumStrahlerOrderValue = MaximumStrahlerOrderTuple[0]
print(MaximumStrahlerOrderValue)

Maintenant nous pouvons boucler (itérer) sur les ordres de Strahler et sauvegarder chaque carte plus grande ou égale à cet ordre.

In [ ]:
for order in range (1,MaximumStrahlerOrderValue + 1):
    Stream = ifthen(StrahlerOrders >= order, boolean(1))
    report(Stream,'stream'+str(order)+'.map')

Visualisez les différents résultats et choisissez celui qui correspond le mieux à OpenStreetMap ou Google Satellite. Pour ce faire, vous pouvez charger les couches dans QGIS.

## 6.2 Méthode d'accumulation de  flux
Afin d'estimer où se trouve la rivière, nous pouvons également utiliser la méthode d'accumulation des flux. Dans PCRaster, cela se fait avec l'opération [accuflux](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_accuflux.html). accumule le matériau (deuxième argument) dans la direction du flux. Ici, nous choisissons 1 pour toutes les cellules comme matériau.

In [ ]:
FlowAccumulation = accuflux(FlowDirection,1)

Visualisez le résultat avec Aguila. Changez la légende en shifted logarithmic, en raison de la large gamme de valeurs.

De la même manière qu'avec la méthode de Strahler, nous devons déterminer l'accumulation minimale de débit que nous considérons comme une rivière. Nous le faisons à nouveau en comparant les cartes booléennes avec une carte de référence. Commençons avec 9000 pixels.

In [ ]:
RiverFlow = ifthen(FlowAccumulation > 9000, boolean(1))

Essayez avec différentes valeurs et vérifiez si cela correspond bien avec par exemple OpenStreetMap dans QGIS.

Après la calibration, écrivez celle qui correspond le mieux sur le disque avec `report`.

L'étape suivante consiste à délimiter le bassin versant appartenant à un exutoire donné.

<!--NAVIGATION-->
< [Calculate Flow Direction](Flowdirection.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Catchment Delineation](CatchmentDelineation.ipynb) >